Querying the Exposure Log
====

This notebook demonstrates querying the USDF copy of the exposure log database, which stores a list of images that have been flagged during observing.

Author: Colin Slater

Date Updated: 2024-01-18


In [ ]:
import lsst.daf.butler as dafButler
import numpy as np

from sqlalchemy import create_engine, text
from collections import defaultdict

In [2]:
b = dafButler.Butler("/repo/embargo", collections=["LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20231207/w_2023_49/PREOPS-4648"])

In [88]:
engine = create_engine("postgresql+psycopg2://usdf@usdf-summitdb.slac.stanford.edu/exposurelog")

In [85]:
log_visit_detector = set([(x.dataId['exposure'], x.dataId['detector']) for x in b.registry.queryDatasets("isr_log")])
print("Number of ISRs attempted: {:d}".format(len(log_visit_detector)))

calexp_visit_detector = set([(x.dataId['visit'], x.dataId['detector']) for x in b.registry.queryDatasets("calexp")])
print("Number of successful calexps attempted: {:d}".format(len(calexp_visit_detector)))

missing_calexps = set(log_visit_detector - calexp_visit_detector)
missing_visits = [x[0] for x in missing_calexps]
print("Number of unsuccessful processCcd attempts (no resulting calexp): {:d}".format(len(missing_calexps)))



Number of ISRs attempted: 3609
Number of successful calexps attempted: 3404
Number of unsuccessful processCcd attempts (no resulting calexp): 205


In [76]:
missing_visits[:3]

[2023080300575, 2023081500123, 2023081700530]

For efficient querying, this next cell groups together all of the dataIds by day_obs.
The result is a dictionary `seq_nums_per_day_obs` where keys are day_obs, and values are the list of seq_nums for which image processing was attempted.

In [92]:
seq_nums_per_day_obs = defaultdict(list)

all_day_obs = [(x['exposure']//100000, x['exposure'] - (100000*(x['exposure']//100000)))  for x in log_dataIds]
for day_obs, seq_num in all_day_obs:
    seq_nums_per_day_obs[day_obs].append(seq_num)


print("Number of day_obs to query: {:d}".format(len(seqnums_per_dateobs.keys())))

Number of day_obs to query: 35


For each day_obs, select all of the exposure log messages that match the seq_nums we attempted to process.
The result is a dictionary `all_obs_messages` where keys are `day_obs` and values are a tuple of seq_num, exposure_flag, message_text, and "ok" or "Calexp failed" depending on whether we have a calexp available.

In [93]:
all_obs_messages = {}
for day_obs in seq_nums_per_day_obs.keys():
    with engine.connect() as conn:
        this_date = []
        seq_nums = seq_nums_per_day_obs[day_obs]
        result = conn.execute(text(
            "select seq_num, exposure_flag, message_text from message WHERE day_obs = {:d} AND seq_num IN ({:s})".format(day_obs, ",".join([str(x) for x in seq_nums]))))
        for res in result:
            seq_num, flag_string, message_text = res
            exposure = (day_obs * 100000) + seq_num
            this_date.append((seq_num, flag_string, message_text, "Calexp Failed" if exposure in missing_visits else "ok"))
        all_obs_messages[day_obs] = this_date


In [94]:
all_obs_messages

{20231130: [],
 20231127: [],
 20230706: [],
 20230511: [],
 20230803: [],
 20231116: [],
 20230524: [(656, 'questionable', 'High mount jitter. ', 'ok'),
  (697, 'questionable', 'undefined', 'ok'),
  (706, 'questionable', 'undefined', 'ok')],
 20230720: [],
 20231107: [(546,
   'none',
   'Significant motion while exposed. Stars are streaked.',
   'Calexp Failed')],
 20230829: [],
 20231114: [],
 20230718: [],
 20231121: [(195, 'questionable', 'undefined', 'ok')],
 20231205: [],
 20231204: [],
 20231120: [],
 20230928: [(109, 'questionable', 'High mount motion.', 'ok'),
  (110, 'questionable', 'High mount motion.', 'ok'),
  (111, 'questionable', 'High mount motion.', 'Calexp Failed'),
  (112, 'questionable', 'High mount motion.', 'ok'),
  (118, 'questionable', 'High mount motion.', 'ok'),
  (117, 'questionable', 'High mount motion.', 'ok'),
  (116, 'questionable', 'High mount motion.', 'ok'),
  (115, 'questionable', 'High mount motion.', 'ok'),
  (113, 'questionable', 'High mount motio